In [4]:
import os
import threading

import cv2 as cv
import dlib
from deepface import DeepFace
from sklearn.metrics import accuracy_score

2024-04-17 23:06:46.383519: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-17 23:06:46.383573: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-17 23:06:46.384482: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-17 23:06:46.390063: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-17 23:06:47.101552: W tensorflow/compiler/tf2

In [5]:
models = [
    "VGG-Face",
    "Facenet",
    "Facenet512",
    "OpenFace",
    "DeepFace",
    "DeepID",
    "ArcFace",
    "Dlib",
    "SFace",
    "GhostFaceNet",
]
metrics = ["cosine", "euclidean", "euclidean_l2"]
backends = [
    "opencv",
    "ssd",
    "dlib",
    "mtcnn",
    "fastmtcnn",
    "retinaface",
    "mediapipe",
    "yolov8",
    "yunet",
    "centerface",
    "skip",
]

In [6]:
def convert_bb(rect):  # convert dlib coords into opencv
    startX = rect.left()
    startY = rect.top()
    endX = rect.right()
    endY = rect.bottom()
    w = endX - startX
    h = endY - startY
    return (startX, startY, w, h)

In [7]:
detector = dlib.get_frontal_face_detector()  # setup detector

In [7]:
img = cv.imread("test.jpg")

In [11]:
faces = DeepFace.find(
    img_path=img,
    db_path="200classes_data/train",
    model_name=models[1],
    distance_metric=metrics[0],
    detector_backend=backends[2],
    enforce_detection=False,
)
person = (
    faces[0]
    .identity.head(3)
    .apply(lambda x: x.split("/")[2])
    .agg({"identity": "value_counts"})
    .idxmax()[1]
)
print(person)

24-04-17 22:15:21 - ⚠️ Representations for images in 200classes_data/train folder were previously stored in representations_facenet.pkl. If you added new instances after the creation, then please delete this file and call find function again. It will create it again.
24-04-17 22:15:21 - There are 1069 representations found in representations_facenet.pkl
24-04-17 22:15:22 - find function lasts 0.8628864288330078 seconds
Colin_Powell


In [15]:
def accuracy(base_path, db_path):
    counter = 0
    predictions = []
    expected = []
    for folder_name in os.listdir(base_path):
        folder_path = os.path.join(base_path, folder_name)
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            face = DeepFace.find(
                img_path=file_path,
                db_path=db_path,
                model_name=models[1],
                distance_metric=metrics[0],
                detector_backend=backends[2],
                enforce_detection=False,
            )
            try:
                person = (
                    face[0]
                    .identity.head(3)
                    .apply(lambda x: x.split("/")[2])
                    .agg({"identity": "value_counts"})
                    .idxmax()[1]
                )
            except ValueError:
                person = "Stranger"
            predictions.append(person)
            expected.append(folder_name)
            counter += 1
            print(counter)
    return accuracy_score(predictions, expected)

In [23]:
#accuracy_200 = accuracy("200classes_data/test", "200classes_data/train")
#accuracy_150 = accuracy("150classes_data/test", "150classes_data/train")
#accuracy_100 = accuracy("100classes_data/test", "100classes_data/train")
#accuracy_50 = accuracy("50classes_data/test", "50classes_data/train")

In [24]:
accuracy_200

0.9025

In [25]:
accuracy_150

0.93

In [26]:
accuracy_100

0.9466666666666667

In [27]:
accuracy_50

0.96

Точность модели на 200 классов:90.25%   
Точность модели на 150 классов:93%   
Точность модели на 100 классов:94.6%   
Точность модели на 50 классов:96%   


In [19]:
cap = cv.VideoCapture("video.mp4")

In [20]:
# set the size of the recorded video
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
frame_size = (frame_width, frame_height)

In [7]:
writer = cv.VideoWriter("result.mp4", cv.VideoWriter_fourcc(*"DIVX"), 60, frame_size)

OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [9]:
def FaceRecognition(faceNames, faceID, frame):
    faces = DeepFace.find(
        img_path=frame,
        db_path="200classes_data/train",
        model_name=models[1],
        distance_metric=metrics[0],
        detector_backend=backends[2],
        enforce_detection=False,
    )
    try:
        person = (
            faces[0]
            .identity.head(3)
            .apply(lambda x: x.split("/")[2])
            .agg({"identity": "value_counts"})
            .idxmax()[1]
        )
    except ValueError:
        person = None
    faceNames[faceID] = person

In [10]:
def thread_function(faceNames, faceID, frame):
    print("Calling FaceRecognition")  # Отладочный вывод
    FaceRecognition(faceNames, faceID, frame)

In [21]:
def face_tracking_and_recognition(cap):

    frameCounter = 0
    currentFaceID = 0
    faceTrackers = {}
    faceNames = {}

    while True:
        ret, frame = cap.read()

        resultFrame = frame.copy()

        frameCounter += 1
        if frameCounter % 300 == 0:  # start face recognition periodically
            t = threading.Thread(
                target=thread_function,
                args=(faceNames, currentFaceID, frame),
                daemon=True,
            )
            t.start()

        faceIDsToDelete = []  # delete bad trackers
        for faceID in faceTrackers.keys():
            trackingQuality = faceTrackers[faceID].update(frame)
            if trackingQuality < 7:
                faceIDsToDelete.append(faceID)
        for faceID in faceIDsToDelete:
            faceTrackers.pop(faceID, None)

        if (frameCounter % 10) == 0:#start scanning for faces
            gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
            faces = detector(gray)
            faces = [convert_bb(r) for r in faces]

            for x, y, w, h in faces:
                x_bar = x + 0.5 * w
                y_bar = y + 0.5 * h
                matchedFaceID = None

                for faceID in faceTrackers.keys():
                    tracked_position = faceTrackers[faceID].get_position()

                    t_x = int(tracked_position.left())
                    t_y = int(tracked_position.top())
                    t_w = int(tracked_position.width())
                    t_h = int(tracked_position.height())

                    t_x_bar = t_x + 0.5 * t_w
                    t_y_bar = t_y + 0.5 * t_h

                    #if face center in tracked bbox and tracked center in face bbox we approved it
                    if (
                        (t_x <= x_bar <= (t_x + t_w))
                        and (t_y <= y_bar <= (t_y + t_h))
                        and (x <= t_x_bar <= (x + w))
                        and (y <= t_y_bar <= (y + h))
                    ):
                        matchedFaceID = faceID

                if matchedFaceID is None:
                    currentFaceID += 1
                    tracker = dlib.correlation_tracker()
                    tracker.start_track(
                        frame,
                        dlib.rectangle(x - 10, y - 20, x + w + 10, y + h + 20),
                    )

                    faceTrackers[currentFaceID] = tracker
                    t_f = threading.Thread(
                        target=FaceRecognition,
                        args=(faceNames, currentFaceID, frame),
                        daemon=True,
                    )
                    t_f.start()


        for faceID in faceTrackers.keys():
            tracked_position = faceTrackers[faceID].get_position()

            t_x = int(tracked_position.left())
            t_y = int(tracked_position.top())
            t_w = int(tracked_position.width())
            t_h = int(tracked_position.height())

            cv.rectangle(
                resultFrame, (t_x, t_y), (t_x + t_w, t_y + t_h), (0, 255, 0), 2
            )

            if faceID in faceNames.keys():
                cv.putText(
                    resultFrame,
                    faceNames[faceID],
                    (int(t_x + t_w / 2), int(t_y)),
                    cv.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    (255, 255, 255),
                    2,
                )
            else:
                cv.putText(
                    resultFrame,
                    "Detecting",
                    (int(t_x + t_w / 2), int(t_y)),
                    cv.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    (255, 255, 255),
                    2,
                )

        #writer.write(resultFrame)
        cv.imshow("result", resultFrame)
        if cv.waitKey(1) & 0xFF == ord("q"):
            break

In [22]:
face_tracking_and_recognition(cap)
cap.release()
cv.destroyAllWindows()

24-04-17 23:10:35 - ⚠️ Representations for images in 200classes_data/train folder were previously stored in representations_facenet.pkl. If you added new instances after the creation, then please delete this file and call find function again. It will create it again.
24-04-17 23:10:35 - There are 1069 representations found in representations_facenet.pkl
24-04-17 23:10:36 - find function lasts 0.9274899959564209 seconds
